# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

In [9]:
import torch
a, b = torch.Tensor([[1,1],[2,2],[1,1]]), torch.Tensor([[2,4],[2,4],[2,4]])
print("----\na:\n", a)
print("a T:\n", a.t())
print("b:\n", b,"----\n")
print("----\na*b:\n",a*b)
#print(torch.dot(a.t(),b))
print("----\nmm:\n",torch.mm(a.t(), b))
print("----\nmatmul:\n",torch.matmul(a.t(), b))
print("----\nbmm (need to unsqueeze to add a batch dim):\n", torch.bmm(a.t().unsqueeze(0), b.unsqueeze(0)))

----
a:
 tensor([[ 1.,  1.],
        [ 2.,  2.],
        [ 1.,  1.]])
a T:
 tensor([[ 1.,  2.,  1.],
        [ 1.,  2.,  1.]])
b:
 tensor([[ 2.,  4.],
        [ 2.,  4.],
        [ 2.,  4.]]) ----

----
a*b:
 tensor([[ 2.,  4.],
        [ 4.,  8.],
        [ 2.,  4.]])
----
mm:
 tensor([[  8.,  16.],
        [  8.,  16.]])
----
matmul:
 tensor([[  8.,  16.],
        [  8.,  16.]])
----
bmm (need to unsqueeze to add a batch dim):
 tensor([[[  8.,  16.],
         [  8.,  16.]]])


In [10]:
a.shape

torch.Size([3, 2])

In [11]:
b.shape

torch.Size([3, 2])

In [14]:
c = torch.Tensor([[10,20]])
c.shape

torch.Size([1, 2])

In [15]:
a+c

tensor([[ 11.,  21.],
        [ 12.,  22.],
        [ 11.,  21.]])

In [19]:
print(a)
print(c)

tensor([[ 1.,  1.],
        [ 2.,  2.],
        [ 1.,  1.]])
tensor([[ 10.,  20.]])


In [17]:
a.numpy() + c.numpy()

array([[ 11.,  21.],
       [ 12.,  22.],
       [ 11.,  21.]], dtype=float32)

In [3]:
x = torch.randn(())
x

tensor(-0.9258)

In [2]:
from torchvision.models import resnet50
resnet = resnet50(pretrained=True)
print(resnet)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 51241689.09it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [2]:
list(resnet.children())[-2]

AvgPool2d(kernel_size=7, stride=1, padding=0)

In [3]:
resnet.fc.in_features

2048

In [ ]:
from torchvision.models import resnet152
test_net = resnet152(pretrained=True)
test_net = nn.Sequential(*list(test_net.children())[:-2])

In [7]:
imgs = torch.cat([img, img])
imgs.shape

torch.Size([2, 3, 224, 224])

In [8]:
x = test_net(imgs)
x = x.permute(0, 2, 3, 1)
x = x.view(x.size(0), -1, x.size(-1))
x.shape

torch.Size([2, 49, 2048])

In [2]:
import torch
import torch.nn as nn
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
def plot_vector(vector):
    plt.figure(figsize=(20, 1))
    ## TODO extract dimension to plot if needed
    sns.heatmap(vector.data.numpy(), annot=True, cmap=sns.light_palette("orange", as_cmap=True), linewidths=1)
    plt.show()

import torch
import torch.nn as nn
from torchvision import transforms
import sys
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# import data
img = Image.open('progress_example.png')
img = transform_train(img).float()
img = img[0:3, :, :] # 3 channels
img = img.unsqueeze(0) # simulate batch
print("img shape:", img.shape)

# 


class DecoderRNN(nn.Module):
    def __init__(embed_size, hidden_size, vocab_size):
        super(DecoderRNN, self).__init__()
        
        # embedding layer that turns tokens into a vector of a specified size
        self.word_embeddings = nn.Embeddings(vocab_size, embed_size)
        
        # RNN cell - GRU
        self.cell = nn.GRU_cell(embed_size, hidden_size)
        
        # add a FC = produces scores for each token in the vocab, to then generate the output
        self.linear = nn.Linear(hidden_size, vocab_size)

        #############
        # Attention #
        
        # Scoring embedded feature vectors (from encoder)
        
    
    def forward(self, features, captions):
        # create embedded vectors for each token in a sequence
        # -- note: captions should be initialized to <start> token
        embeddings = self.word_embeddings(captions)
        
        
        lstm_out, self.hidden = self.lstm(
            embeddings.view(len(sentence), 1, -1), self.hidden)
        

    def sample(self, inputs, states=None, max_len=20):
        " accepts pre-processed image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "
        for i in range(max_len):
             pass
        
from model import EncoderCNN, Attention

#forward pass in Encoder
net = EncoderCNN(embed_size)
feature_vectors = net(img)
print("feature_vectors:", feature_vectors.shape)

# fake decoder hidden state
hid = torch.ones(1, hidden_size).unsqueeze(1)
print("dec hidden state:", hid.shape)

# compute attention vector
att = Attention(net.feature_dim, hidden_size, embed_size)
context = att(feature_vectors, hid)
print("context vector shape", context.shape)
print("context vector...", context[0, :10])

img shape: torch.Size([1, 3, 224, 224])
feature_vectors: torch.Size([1, 49, 512])
dec hidden state: torch.Size([1, 1, 512])
context vector shape torch.Size([1, 512])
context vector... tensor([-0.2779,  0.0643,  0.4330,  0.4619,  0.0786, -0.0636,  0.1112,
        -0.0695, -0.1564,  0.4502])


<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

*Architecture*

**Encoder**: **Resnet50 pretrained on ImageNet stripped of last layer + linear layer** 
- ResNet50 outperforms ResNet152 (and has less layers). Authors in [1] and [2] used VGG16 as the ResNet had not been published. However, since being published ResNet has consistentlty outperformed VGG on the annual ImageNet benchmark. I hence picked it to provide better embeddings to my decoder
- I strip the last layer (used to classify images) and pass the feature map to a new linear layer to produce embeddings of the desired size for the decoder.

**Attention**: **Concat twist on the Bahdanau attention**
- `score = V tanh(U hidden + W feature_vectors)`
- `weights = softmax(score)`
- `context_vector = (weights * feature_vectors).sum(1)`

**Decoder**: **GRU with teacher forcing**
- GRU: performs slightly better than LSTM for image captioning, with less parameters [3].
- Teacher forcing: use the ground truth token (t-1) instead of predicted output (t-1) as input for step (t), as presented in [4].
- Context vector concatenated with previous output (here embedded ground truth token) as input to the GRU, as well as the previous hidden state.

*References*

*[1] Show and Tell: A Neural Image Caption Generator, Vinyals et al, 2015.*

*[2] Show, Attend and Tell: Neural Image Caption Generation with Visual Attention, Xu et al, 2016*

*[3] An Empirical Study of Language CNN for Image Captioning, Gu et al, 2017. [https://arxiv.org/pdf/1612.07086.pdf ]*

*[4] Teacher Forxing, section 10.2.1, Deep Learning, Courville Goodfellow Bengio, 2015. [https://www.deeplearningbook.org/contents/rnn.html ]*

*[5] Show, Attend, and Tell | a PyTorch Tutorial to Image Captioning, Sagar Vinodababu, 2018. [https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning ]

*[6] Show, Attend and Tell, Aaron Wong, 2018. [https://github.com/AaronCCWong/Show-Attend-and-Tell ]*

- batch_size=**64** as recommended in [2]
- vocab_threshold=**5** as recommended in [1]
- vocab_from_file=**True** to speed up the instantiation of the data loader
- embed_size=**512** as recommended in [1] and [2]
- hidden_size=**512** as recommended in [1] and [2]
- num_epochs=**3** as recommended here
- save_every=**1** as recommended here
- print_every=**100** as recommended here


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that the images in the dataset have varying heights and widths, and if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

I kept the transformations as is. Normalization helps with training, Resizing helps feeding the right input shape to the network. RandomCrop and RandomFlip are both useful data augmentation techniques to virtually augment the size of the training set. Particularly, a horizontal flip (left/right) doesn't hurt the semantic content for scene description, apart for specific positioning adverbs ("left of", "on his right"...).

I **did NOT add a vertical flip** (up/down) as it would mess with training in affecting the semantic content (eg: sky/grass upside-down could be taken as a body of water in a park).

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:**

`list(decoder.parameters()) + list(encoder.fc.parameters())`

I am using a pretrained resnet for the encoder and do not wish to finetune its weight. Only the **encoder last layer** (FC = embeddings) needs to be trained. Not finetuning is also the method adopted in [2]:

"we used [the Oxford VGGnet] pre-trained on ImageNet **without finetuning**"

The **decoder parameters** are initialized randomly and need to be trained (no pretrained network used). I did not explicitely specify the **attention parameters**, as those are included in the decoder parameters already (attention is passed as an attribute to the decoder -- to check: `[i[0] for i in list(decoder.named_parameters())]`)

*[2] Show, Attend and Tell: Neural Image Caption Generation with Visual Attention, Xu et al, 2016*

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

Even though [1] uses SGD with no momentum and a fixed learning rate, the authors of [2] (more recent, iteration on [1]) recommend instead **Adam** (SGD with adaptative LR).

"Both variants of our attention model were trained with stochastic gradient descent using adaptive learning rate algorithms. For the Flickr8k dataset, we found that RMSProp (Tieleman & Hinton, 2012) worked best, while **for** Flickr30k/**MS COCO dataset we used the recently proposed Adam algorithm** (Kingma & Ba, 2014)."

*[2] Show, Attend and Tell: Neural Image Caption Generation with Visual Attention, Xu et al, 2016*



In [2]:
import nltk
nltk.download('punkt')

import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN, Attention
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...



100%|██████████| 414113/414113 [01:37<00:00, 4253.49it/s]

Done (t=0.93s)
creating index...
index created!
Obtaining caption lengths...


In [71]:
%reload_ext autoreload
%autoreload 2
from model import EncoderCNN, DecoderRNN, Attention
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the encoder and decoder.
encoder = EncoderCNN(embed_size)
attention = Attention(encoder.feature_dim, hidden_size, embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, attention, device)

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

In [72]:
[i[0] for i in list(decoder.named_parameters())]

['word_embeddings.weight',
 'attention.U.weight',
 'attention.U.bias',
 'attention.W.weight',
 'attention.W.bias',
 'attention.V.weight',
 'attention.V.bias',
 'fc_init_hidden.weight',
 'fc_init_hidden.bias',
 'fc_init_output.weight',
 'fc_init_output.bias',
 'gru.weight_ih_l0',
 'gru.weight_hh_l0',
 'gru.bias_ih_l0',
 'gru.bias_hh_l0',
 'fc_out.weight',
 'fc_out.bias']

In [73]:
import torch.optim as optim
# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.fc.parameters())

# TODO #4: Define the optimizer.
optimizer = optim.Adam(params, lr=0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [74]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
#         print("\n---------\nCaptions")
#         print(captions.shape)
#         print(captions[0])
#         print("\n---------\nOutput")
#         print(outputs.shape)
#         print(outputs[0][0])
        
#         print("\n---------\nCaptions view")
#         print(captions.to('cpu').data.view(-1).shape)
#         print(captions.to('cpu').data.view(-1)[:5])
        
#         print("\n---------\nOutputs view")
#         print(outputs.view(-1, vocab_size).shape)
#         print(outputs.view(-1, vocab_size)[0][:5])
#         print(outputs.view(-1, vocab_size)[0].max(0))

                
         # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.to('cpu').data.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-v2-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-v2-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 4.5896, Perplexity: 98.45549
Epoch [1/3], Step [200/6471], Loss: 4.2979, Perplexity: 73.54822
Epoch [1/3], Step [300/6471], Loss: 4.0035, Perplexity: 54.79140
Epoch [1/3], Step [400/6471], Loss: 3.7876, Perplexity: 44.15213
Epoch [1/3], Step [500/6471], Loss: 3.8405, Perplexity: 46.54648
Epoch [1/3], Step [600/6471], Loss: 3.6326, Perplexity: 37.8128
Epoch [1/3], Step [700/6471], Loss: 3.4805, Perplexity: 32.4771
Epoch [1/3], Step [800/6471], Loss: 3.4686, Perplexity: 32.0914
Epoch [1/3], Step [900/6471], Loss: 3.1341, Perplexity: 22.9690
Epoch [1/3], Step [1000/6471], Loss: 3.8045, Perplexity: 44.9019
Epoch [1/3], Step [1100/6471], Loss: 3.3552, Perplexity: 28.6520
Epoch [1/3], Step [1200/6471], Loss: 3.6322, Perplexity: 37.7961
Epoch [1/3], Step [1300/6471], Loss: 3.1377, Perplexity: 23.0501
Epoch [1/3], Step [1400/6471], Loss: 3.6258, Perplexity: 37.5545
Epoch [1/3], Step [1500/6471], Loss: 3.0039, Perplexity: 20.1648
Epoch [1/3], Step [1600/6471]

KeyboardInterrupt: 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.